In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
def get_dataset(path):
    tokens = []
    ner_tags = []
    token = []
    ner_tag = []

    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            # 1. Clean up and check if the line is empty (end of sentence)
            line = line.strip()
            if not line:
                # Only append if the temporary lists are not empty (fixes the bug of appending empty lists)
                if token:
                    tokens.append(token)
                    ner_tags.append(ner_tag)

                # Reset temporary lists
                token = []
                ner_tag = []
                continue

            # 2. Process token and tag for non-empty lines
            # Example line format: Word POS-Tag Chunk-Tag NER-Tag
            fields = line.split()

            # Defensive check: ensure there are enough fields (at least 2: token and tag)
            if len(fields) >= 2:
                t = fields[0]    # The token (always the first field)
                l = fields[-1]   # The NER tag (always the last field)

                token.append(t)
                ner_tag.append(l)

    # 3. Handle the last sentence in the file (CRITICAL FIX)
    # If the file does not end with a blank line, the last sentence needs to be appended
    if token:
        tokens.append(token)
        ner_tags.append(ner_tag)

    print(len(tokens), len(ner_tags))
    print(tokens[0], ner_tags[0])

    return {
        'tokens': tokens,
        'ner_tags': ner_tags
    }

# 👇 FIX IS HERE: Change the file extensions from '.conll' to '.txt'
raw_train_data = get_dataset('train.txt')
raw_dev_data = get_dataset('valid.txt')
raw_test_data = get_dataset('test.txt')

14987 14987
['-DOCSTART-'] ['O']
3466 3466
['-DOCSTART-'] ['O']
3684 3684
['-DOCSTART-'] ['O']


In [ ]:
!ls

sample_data


# Mục mới

In [ ]:
from pprint import pprint
label_dict = {}
for i in raw_train_data['ner_tags']:
    for j in i:
        if j not in label_dict.keys():
            label_dict[j]=len(label_dict)


label_dict['I-email'] = len(label_dict)
pprint(label_dict)
reverse_label_dict = {v:k for k,v in label_dict.items()}


{'B-LOC': 5,
 'B-MISC': 2,
 'B-ORG': 1,
 'B-PER': 3,
 'I-LOC': 8,
 'I-MISC': 7,
 'I-ORG': 6,
 'I-PER': 4,
 'I-email': 9,
 'O': 0}


In [ ]:
import torch
class TrainData():
    def __init__(self,input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
    def __getitem__(self,idx):
        return [
            torch.tensor(self.input_ids[idx]),
            torch.tensor(self.attention_mask[idx]),
            torch.tensor(self.labels[idx])
        ]
    def __len__(self):
        return len(self.labels)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer('hola', add_special_tokens=True))
tokenizer('hola', add_special_tokens=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

{'input_ids': [0, 739, 143, 2], 'attention_mask': [1, 1, 1, 1]}


{'input_ids': [739, 143], 'attention_mask': [1, 1]}

In [ ]:
ignore_index =-100
max_length=128
batch_size = 32

def pad_to_maxlength(l,max_length, pad_value):
    if len(l) < max_length:
        l+=[pad_value]*(max_length-len(l))
    else:
        l = l[:max_length]
    return l
def get_training_data(data, shuffle=True):
    # print(data.data['tokens'])
    tokens = []
    max_l = 0
    for i in data['tokens']:
        if i ==[]:
            print('empty')
        tokens.append(i)
        if len(i)> max_l:
            max_l = len(i)
    print(max_l)
    print('Checking')
    raw_labels = data['ner_tags']
    # for i in range(len(raw_labels)):
    #     for j in range(len(raw_labels[i])):
    #         try:
    #             raw_labels[i][j]= label_dict[raw_labels[i][j]]
    #         except:
    #             print(raw_labels[i][j])
    #             return
    raw_labels= [i for i in raw_labels if i!= []]
    labels = []

    tokens = [i for i in tokens if i!= []]
    input_ids = []
    attention_masks = []
    print(len(tokens),len(raw_labels))
    assert len(tokens) == len(raw_labels)
    for i,token in enumerate(tokens):

        tmp_input_ids = [tokenizer.cls_token_id]
        tmp_attention_mask = [1]
        tmp_labels = ['O']
        for token_index, t in enumerate(token):
            t_encoded = tokenizer(t, add_special_tokens=False)
            tmp_input_ids +=t_encoded['input_ids']
            tmp_label = [raw_labels[i][token_index]]*len(t_encoded['input_ids'])
            for label_idx, label_token in enumerate(tmp_label):
                if label_idx != 0 and label_token[0] == 'B':
                    label_token = 'I'+label_token[1:]
                    tmp_label[label_idx] = label_token
            tmp_labels += tmp_label
            tmp_attention_mask+= t_encoded['attention_mask']
        tmp_input_ids+=[tokenizer.sep_token_id]
        tmp_attention_mask += [1]
        tmp_labels += ['O']
        tmp_input_ids = pad_to_maxlength(tmp_input_ids, max_length,tokenizer.pad_token_id)
        tmp_attention_mask = pad_to_maxlength(tmp_attention_mask, max_length, 0)
        tmp_labels = [label_dict[i] for i in tmp_labels]
        tmp_labels = pad_to_maxlength(tmp_labels, max_length, ignore_index)


        input_ids.append(tmp_input_ids)
        attention_masks.append(tmp_attention_mask)
        labels.append(tmp_labels)
    print(input_ids[-1][:len(token)+10], labels[-1][:len(token)+10], sep ='\n')







    data_class= TrainData(
        input_ids,
        attention_masks,
        labels
    )


    loader = DataLoader(data_class, batch_size=batch_size,shuffle=shuffle )
    return loader

train_data = get_training_data(raw_train_data)
dev_data = get_training_data(raw_dev_data, shuffle=False)
test_data = get_training_data(raw_test_data, shuffle=False)

113
Checking
14987 14987
[0, 138087, 184, 11, 106, 138319, 116, 2, 1, 1, 1, 1, 1, 1]
[0, 1, 6, 6, 0, 1, 0, 0, -100, -100, -100, -100, -100, -100]
109
Checking
3466 3466
[0, 4210, 111617, 1833, 28905, 6, 111056, 5428, 42221, 173300, 363, 2, 1, 1]
[0, 0, 1, 6, 6, 0, 0, 0, 0, 0, 0, 0, -100, -100]
124
Checking
3684 3684
[0, 581, 21, 29188, 36770, 636, 297, 7, 14098, 86841, 6777, 959, 3249, 1919, 30715, 34377, 24189, 70, 32070, 111, 496, 1284, 155605, 23742, 2273, 3540, 7, 136, 509, 10, 22799, 32786, 111, 70, 36236, 6661, 15619, 6, 69986, 7175, 678, 1919, 27150, 56, 82953, 6, 4, 163359, 6, 5, 2]
[0, 0, 0, 0, 0, 1, 6, 6, 6, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0]


In [ ]:
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=fc000721f6ca10645860d6eb31f59c2b8e97244caef17f2758b62b1d4925e3f6
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [ ]:
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from seqeval.metrics import classification_report
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR

n_epochs=1
lr = 2e-5
log_step =10
eval_step =200
total_steps = n_epochs*len(train_data)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_dict))
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = LinearLR(optimizer,total_iters=total_steps)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

## train_loop


if torch.cuda.is_available():
    device ='cuda'
else:
    device = 'cpu'

model.to(device)
step =0
avg_loss = 0
best_f1 = 0
for ep in range(n_epochs):
    for i, batch in enumerate(train_data):
        model.train()

        input_ids, attention_mask,label =batch

        optimizer.zero_grad()
        out = model(input_ids.to(device), attention_mask.to(device)).logits
        out = out.permute(0,-1,1)
        # print(out.shape, label.shape)
        # print(out[0], label[0], attention_mask[0], input_ids[0])


        loss = loss_fn(out, label.to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()
        avg_loss += loss.item()
        if step % eval_step ==0 or step == total_steps -1:
            if step == total_steps -1:
                model.load_state_dict(best_state_dict)
            model.eval()
            y_true = []
            y_pred = []
            with torch.no_grad():
                for ii, e_batch in enumerate(dev_data):

                    input_ids, attention_mask,labels = e_batch
                    for y in labels:
                        y = [reverse_label_dict[i.item()] for i in y if i != ignore_index]
                        y_true.append(y)
                    pred = model(input_ids.to(device), attention_mask.to(device)).logits
                    # pred = pred.permute(0,-1,1)
                    pred = torch.argmax(pred,dim=-1)
                    pred = pred.detach().cpu()
                    for idx,p in enumerate(pred):
                        p = [reverse_label_dict[p[i].item()] for i in range(len(p)) if input_ids[idx][i]!=tokenizer.pad_token_id]
                        y_pred.append(p)

            print(classification_report(y_true,y_pred, digits=4, zero_division=0))
            f1 = classification_report(y_true,y_pred,digits=4, zero_division=0, output_dict = True)['weighted avg']['f1-score']
            if f1 > best_f1:
                best_f1 = f1
                best_state_dict = model.state_dict()
        if step % log_step == 0:
            print('Step {}/{}, Loss {} Best F1: {}'.format(step,total_steps,avg_loss/log_step, best_f1))
            avg_loss = 0

        step+=1





              precision    recall  f1-score   support

         LOC     0.9639    0.9592    0.9615      1837
        MISC     0.8383    0.8937    0.8651       922
         ORG     0.8677    0.9045    0.8857      1341
         PER     0.9621    0.9662    0.9641      1837

   micro avg     0.9207    0.9389    0.9297      5937
   macro avg     0.9080    0.9309    0.9191      5937
weighted avg     0.9221    0.9389    0.9302      5937

Step 0/469, Loss 0.0012949813157320023 Best F1: 0.930241168924604
Step 10/469, Loss 0.044413292035460475 Best F1: 0.930241168924604
Step 20/469, Loss 0.04724767319858074 Best F1: 0.930241168924604
Step 30/469, Loss 0.040801858576014635 Best F1: 0.930241168924604
Step 40/469, Loss 0.03440300710499287 Best F1: 0.930241168924604
Step 50/469, Loss 0.05269299857318401 Best F1: 0.930241168924604
Step 60/469, Loss 0.04285178724676371 Best F1: 0.930241168924604
Step 70/469, Loss 0.04143233615905047 Best F1: 0.930241168924604
Step 80/469, Loss 0.04672691440209746 Best

In [ ]:
# model.load_state_dict(best_state_dict)
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    tokens = []
    for ii, e_batch in enumerate(test_data):


        input_ids, attention_mask,labels = e_batch
        for i in input_ids:
            tmp = [jj for jj in i if jj !=1]
            # tmp = tmp[1:-1] ## bỏ cls, sep
            tokens.append(tmp)
        for y in labels:
            y = [reverse_label_dict[i.item()] for i in y if i != ignore_index]
            y_true.append(y)
        pred = model(input_ids.to(device), attention_mask.to(device)).logits
        # pred = pred.permute(0,-1,1)
        pred = torch.argmax(pred,dim=-1)
        pred = pred.detach().cpu()
        for idx,p in enumerate(pred):
            p = [reverse_label_dict[p[i].item()] for i in range(len(p)) if labels[idx][i]!=ignore_index]
            y_pred.append(p)

print(classification_report(y_true,y_pred, digits=4, zero_division=0))


### lưu lại

print(len(tokens),len(y_true),len(y_pred))

              precision    recall  f1-score   support

         LOC     0.9105    0.9285    0.9194      1665
        MISC     0.7192    0.7991    0.7571       702
         ORG     0.8579    0.8904    0.8739      1661
         PER     0.9576    0.9517    0.9546      1614

   micro avg     0.8822    0.9078    0.8948      5642
   macro avg     0.8613    0.8924    0.8762      5642
weighted avg     0.8847    0.9078    0.8959      5642

3684 3684 3684


In [ ]:
for i in range(len(tokens)):
    if len(tokens[i])!= len(y_pred[i]):
        print('Error on {}'.format(i))

with open('test_results.txt','w') as f:
    for i in range(len(tokens)):
        for t, lab, pred in zip(tokens[i],y_true[i],y_pred[i]):
            f.write('\t'.join([tokenizer.decode(t),lab,pred,'\n']))
            f.write('\n')


In [ ]:
## Examples
tmp_text = 'i am rishabh i know java c++'
label = ['O','B-name','O','O','O','O','O','B-skill','I-skill','O','O','O','O','B-skill','O','O','B-skill','O','B-skill','I-skill','O']
print(len(tokenizer(tmp_text)['input_ids']), len(label))
print(label)
tokenized = tokenizer(tmp_text)['input_ids']
for i in range(len(tokenized)):
    print(tokenizer.decode(tokenized[i]), label[i])

13 21
['O', 'B-name', 'O', 'O', 'O', 'O', 'O', 'B-skill', 'I-skill', 'O', 'O', 'O', 'O', 'B-skill', 'O', 'O', 'B-skill', 'O', 'B-skill', 'I-skill', 'O']
<s> O
i B-name
am O
 O
rish O
abh O
i O
know B-skill
ja I-skill
va O
c O
++ O
</s> O


In [ ]:
def get_entity(tokens, labels):
    i = 0
    tag_list = []
    found = False
    tmp_token_list = []
    tmp_label = 'O'
    while (i < len(tokens)):
        # print(i, tokens[i], labels[i])


        if labels[i].startswith('B') and not found:
            tmp_token_list.append(tokens[i])
            tmp_label = labels[i][2:]
            found = True
            i+=1
        if found:
            if labels[i] == 'O':
                tag_list.append((tokenizer.decode(tmp_token_list), tmp_label))
                tmp_token_list = []
                # print(tag_list[-1])
                tmp_label='O'
                found = False
            elif labels[i].startswith('I') and labels[i][2:]==tmp_label:
                tmp_token_list.append(tokens[i])
            elif labels[i].startswith('B'):
                tag_list.append((tokenizer.decode(tmp_token_list), tmp_label))
                tmp_token_list = []
                tmp_token_list.append(tokens[i])
                tmp_label = labels[i][2:]
                found = False
        i+=1
    if len(tmp_token_list) > 0:
        tag_list.append((tokenizer.decode(tmp_token_list), tmp_label))

    print(tag_list)
print(tmp_text)
get_entity(tokenized,label)


i am rishabh i know java c++
[('i', 'name'), ('know ja', 'skill')]
